### taeyoonahn

**1. 연금보험료 측면 기존 유니콘 기업 데이터 분석**   
**&nbsp;&nbsp;1.1. 유니콘으로 인정받은 시점의 연금보험료 최소값     
&nbsp;&nbsp;1.2. 기업별 기간내 연금보험료 평균 상승율    
&nbsp;&nbsp;1.3. 유니콘으로 인정받은 시점의 매출/연금보험료 최소값    
&nbsp;&nbsp;1.4. 기업별 기간내 매출/연금보험료 상승율**

**2. 최종 산출물**   
**&nbsp;&nbsp;2.1. 유니콘으로 인정받은 시점의 연금보험료 최소값     
&nbsp;&nbsp;2.2. 기업별 기간내 연금보험료 평균 상승율   
&nbsp;&nbsp;2.3. 유니콘으로 인정받은 시점의 매출/연금보험료 최소값        
&nbsp;&nbsp;2.4. 기업별 기간내 매출/연금보험료 상승율**

**-연금보험료 : 연도별 월 평균 연금보험료를 뜻함**

In [149]:
#package 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family="Malgun Gothic")

from datetime import datetime
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings(action='ignore')

In [150]:
#실수는 소수점 두자리까지 표기
pd.options.display.float_format = '{:.2f}'.format

In [151]:
#데이터 불러오기
df_ori=pd.read_csv('unicorn.csv',engine = "python", encoding = "CP949")
df_ori

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
0,지피클럽,22474017,2015,11,733104.00,8
1,지피클럽,22474017,2015,12,745068.00,7
2,지피클럽,48329929,2016,1,551118.00,7
3,지피클럽,48329929,2016,2,488133.00,7
4,지피클럽,48329929,2016,3,1073632.00,7
...,...,...,...,...,...,...
545,옐로모바일,250700000,2019,8,1981617.00,11
546,옐로모바일,250700000,2019,9,1805970.00,11
547,옐로모바일,250700000,2019,10,1878560.00,11
548,옐로모바일,250700000,2019,11,2212953.00,12


In [152]:
#데이터 정보 확인
print(df_ori.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회사ID      550 non-null    object 
 1   연매출액      550 non-null    int64  
 2   년도        550 non-null    int64  
 3   월         550 non-null    int64  
 4   월별_연금보험료  550 non-null    float64
 5   월별_직원수    550 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 25.9+ KB
None


In [153]:
#데이터 결측치 확인
df_ori.isnull()
df_ori.isnull().sum()

회사ID        0
연매출액        0
년도          0
월           0
월별_연금보험료    0
월별_직원수      0
dtype: int64

In [154]:
#연금보험료 및 연매출액을 다루기 쉽게 데이터프레임 변경
df=df_ori[['회사ID','년도','월별_연금보험료','연매출액']]
df

,회사ID,년도,월별_연금보험료,연매출액
0,지피클럽,2015,733104.00,22474017
1,지피클럽,2015,745068.00,22474017
2,지피클럽,2016,551118.00,48329929
3,지피클럽,2016,488133.00,48329929
4,지피클럽,2016,1073632.00,48329929
...,...,...,...,...
545,옐로모바일,2019,1981617.00,250700000
546,옐로모바일,2019,1805970.00,250700000
547,옐로모바일,2019,1878560.00,250700000
548,옐로모바일,2019,2212953.00,250700000


In [155]:
#연금보험료를 다루기 쉽게 데이터프레임 변경 
#월별 연금보험료의 결측치를 고려해 연도별 월 평균 연금보험료로 바꿈
df_year = df.groupby(['회사ID','년도']).mean()
df_year.reset_index(inplace=True)
df_year.rename(columns = {'월별_연금보험료' : '연도별_월평균연금보험료'}, inplace = True)
df_year

,회사ID,년도,연도별_월평균연금보험료,연매출액
0,무신사,2015,5921860.50,32930316
1,무신사,2016,7283040.33,47231240
2,무신사,2017,9954748.00,67721259
3,무신사,2018,16969701.42,105012384
4,무신사,2019,36105729.83,210312756
5,쏘카,2015,9692472.47,44800000
6,쏘카,2016,17036747.99,88240403
7,쏘카,2017,25767927.26,121086079
8,쏘카,2018,29599275.34,159424490
9,쏘카,2019,42750757.14,256671600


**1. 연금보험료 측면 기존 유니콘 기업 데이터 분석**

**1.1. 기업별 유니콘으로 인정받은 시점의 연금보험료 최소값**

> **기업별 유니콘으로 인정받은 시점**   
>
>**-무신사 2019.11   
-쏘카 2020.10   
-야놀자 2019.6   
-에이프로젠 2019.5   
-엘앤피코스메틱 2016.1   
-옐로모바일 2014.11   
-우아한형제들 2014.12   
-지피클럽 2018.10   
-쿠팡 2014.5   
-크래프톤 2018.8   
-토스 2018.12**

In [156]:
df_unicorn=pd.DataFrame(data={'회사ID':['무신사','쏘카','야놀자','에이프로젠','엘앤피코스메틱','옐로모바일','우아한형제들','지피클럽','쿠팡','크래프톤','토스'],'유니콘_등재년도':['2019','2020','2019','2019','2016','2014','2014','2018','2014','2018','2018']})

> **기업별 유니콘으로 인정받은 시점의 연금보험료**   
>
>**-무신사 36105729.83 
-쏘카 알수없음   
-야놀자 95933831.73   
-에이프로젠 18982074.43   
-엘앤피코스메틱 20327331.33   
-옐로모바일 5167708.50(2015)   
-우아한형제들 293398073.10(2015)   
-지피클럽 3796700.42   
-쿠팡 731149356.50(2015)   
-크래프톤 80043122.83   
-토스 247811184.05**

**1.2. 기업별 기간내 연금보험료 평균 상승율**

In [157]:
#기업별 기간데이터 추출
df_period=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.max())-(df_year.sort_values(['년도']).groupby(['회사ID']).년도.min())
df_period=pd.DataFrame(df_period).reset_index()
df_period.rename(columns={'년도':'기간'},inplace=True)
df_period

,회사ID,기간
0,무신사,4
1,쏘카,4
2,야놀자,4
3,에이프로젠,4
4,엘앤피코스메틱,4
5,옐로모바일,4
6,우아한형제들,4
7,지피클럽,4
8,쿠팡,4
9,크래프톤,4


In [158]:
#기업별 연도별_월평균보험료 변화량 추출-max년도
df_pen_max=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.max())
df_pen_max=pd.DataFrame(df_pen_max).reset_index()
df_pen_max=pd.merge(df_pen_max,df_year)
df_pen_max.rename(columns = {'연도별_월평균연금보험료' : 'max년도_월평균연금보험료'}, inplace = True)
df_pen_max.rename(columns = {'년도' : '년도max'}, inplace = True)
df_pen_max.rename(columns = {'연매출액' : 'max년도_연매출액'}, inplace = True)
df_pen_max

,회사ID,년도max,max년도_월평균연금보험료,max년도_연매출액
0,무신사,2019,36105729.83,210312756
1,쏘카,2019,42750757.14,256671600
2,야놀자,2019,95933831.73,151000000
3,에이프로젠,2019,18982074.43,17857139
4,엘앤피코스메틱,2019,35541135.17,178190156
5,옐로모바일,2019,2675334.42,250700000
6,우아한형제들,2019,1473860019.92,561133434
7,지피클럽,2019,11736785.00,448642100
8,쿠팡,2019,1089015207.33,7140720000
9,크래프톤,2019,118884283.75,12102432


In [159]:
#기업별 연도별_월평균보험료 변화량 추출-min년도
df_pen_min=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.min())
df_pen_min=pd.DataFrame(df_pen_min).reset_index()
df_pen_min=pd.merge(df_pen_min,df_year)
df_pen_min.rename(columns = {'연도별_월평균연금보험료' : 'min년도_월평균연금보험료'}, inplace = True)
df_pen_min.rename(columns = {'년도' : '년도min'}, inplace = True)
df_pen_min.rename(columns = {'연매출액' : 'min년도_연매출액'}, inplace = True)
df_pen_min

,회사ID,년도min,min년도_월평균연금보험료,min년도_연매출액
0,무신사,2015,5921860.50,32930316
1,쏘카,2015,9692472.47,44800000
2,야놀자,2015,15675452.62,29899218
3,에이프로젠,2015,9297395.32,1823518
4,엘앤피코스메틱,2015,12470524.50,188859253
5,옐로모바일,2015,5167708.50,318200000
6,우아한형제들,2015,293398073.10,49500000
7,지피클럽,2015,739086.00,22474017
8,쿠팡,2015,731149356.50,1133752000
9,크래프톤,2015,35047235.00,46400000


In [160]:
#기업별 연도별_월평균보험료 변화량 추출
df_pen_diff=pd.merge(df_pen_min,df_pen_max)
df_pen_diff['변화비율']=df_pen_diff['min년도_월평균연금보험료']/df_pen_diff['max년도_월평균연금보험료']
df_pen_diff=df_pen_diff.drop(['년도min','년도max','max년도_월평균연금보험료','min년도_월평균연금보험료','min년도_연매출액','max년도_연매출액'],axis=1)
df_pen_diff

,회사ID,변화비율
0,무신사,0.16
1,쏘카,0.23
2,야놀자,0.16
3,에이프로젠,0.49
4,엘앤피코스메틱,0.35
5,옐로모바일,1.93
6,우아한형제들,0.20
7,지피클럽,0.06
8,쿠팡,0.67
9,크래프톤,0.29


In [161]:
df_pen_dif=pd.merge(df_pen_diff,df_period)
df_pen_dif['기간']=df_pen_dif['기간'].astype(float)
df_pen_dif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사ID    11 non-null     object 
 1   변화비율    11 non-null     float64
 2   기간      11 non-null     float64
dtypes: float64(2), object(1)
memory usage: 352.0+ bytes


In [163]:
df_pen_dif['CAGR']=(df_pen_dif['변화비율'])**(1/df_pen_dif['기간'])-1
df_pen_cagr=df_pen_dif
df_pen_cagr

,회사ID,변화비율,기간,CAGR
0,무신사,0.16,4.00,-0.36
1,쏘카,0.23,4.00,-0.31
2,야놀자,0.16,4.00,-0.36
3,에이프로젠,0.49,4.00,-0.16
4,엘앤피코스메틱,0.35,4.00,-0.23
5,옐로모바일,1.93,4.00,0.18
6,우아한형제들,0.20,4.00,-0.33
7,지피클럽,0.06,4.00,-0.50
8,쿠팡,0.67,4.00,-0.09
9,크래프톤,0.29,4.00,-0.26


**1.3. 유니콘으로 인정받은 시점의 매출/연금보험료 최소값**

In [129]:
df_spp=df_year
df_spp['연매출액/연도별_월평균연금보험료']=df_spp['연매출액']/df_spp['연도별_월평균연금보험료']
df_spp

,회사ID,년도,연도별_월평균연금보험료,연매출액,연매출액/연도별_월평균연금보험료
0,무신사,2015,5921860.50,32930316,5.56
1,무신사,2016,7283040.33,47231240,6.49
2,무신사,2017,9954748.00,67721259,6.80
3,무신사,2018,16969701.42,105012384,6.19
4,무신사,2019,36105729.83,210312756,5.82
5,쏘카,2015,9692472.47,44800000,4.62
6,쏘카,2016,17036747.99,88240403,5.18
7,쏘카,2017,25767927.26,121086079,4.70
8,쏘카,2018,29599275.34,159424490,5.39
9,쏘카,2019,42750757.14,256671600,6.00


> **기업별 유니콘으로 인정받은 시점**   
>
>**-무신사 2019.11   
-쏘카 2020.10   
-야놀자 2019.6   
-에이프로젠 2019.5   
-엘앤피코스메틱 2016.1   
-옐로모바일 2014.11   
-우아한형제들 2014.12   
-지피클럽 2018.10   
-쿠팡 2014.5   
-크래프톤 2018.8   
-토스 2018.12**

> **기업별 유니콘으로 인정받은 시점의 시점의 매출/연금보험료**   
>
>**-무신사 5.82   
-쏘카 알수없음   
-야놀자 1.57   
-에이프로젠 0.94   
-엘앤피코스메틱 19.49   
-옐로모바일 61.57(2015)  
-우아한형제들 0.17(2015)   
-지피클럽 135.31    
-쿠팡 1.55(2015)      
-크래프톤 13.36     
-토스 0.22**

**1.4. 기업별 기간내 매출/연금보험료 상승율**

In [141]:
#기업별 연도별_월평균보험료 변화량 추출-max년도
df_spp_max=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.max())
df_spp_max=pd.DataFrame(df_spp_max).reset_index()
df_spp_max=pd.merge(df_spp_max,df_year)
df_spp_max.drop(['연도별_월평균연금보험료','연매출액'],axis=1,inplace=True)
df_spp_max.rename(columns = {'연매출액/연도별_월평균연금보험료' : 'max년도_연매출액/월평균연금보험료'}, inplace = True)
df_spp_max.rename(columns = {'년도' : '년도max'}, inplace = True)
df_spp_max

,회사ID,년도max,max년도_연매출액/월평균연금보험료
0,무신사,2019,5.82
1,쏘카,2019,6.00
2,야놀자,2019,1.57
3,에이프로젠,2019,0.94
4,엘앤피코스메틱,2019,5.01
5,옐로모바일,2019,93.71
6,우아한형제들,2019,0.38
7,지피클럽,2019,38.23
8,쿠팡,2019,6.56
9,크래프톤,2019,0.10


In [143]:
#기업별 연도별_월평균보험료 변화량 추출-min년도
df_spp_min=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.min())
df_spp_min=pd.DataFrame(df_spp_min).reset_index()
df_spp_min=pd.merge(df_spp_min,df_year)
df_spp_min.drop(['연도별_월평균연금보험료','연매출액'],axis=1,inplace=True)
df_spp_min.rename(columns = {'연매출액/연도별_월평균연금보험료' : 'min년도_연매출액/월평균연금보험료'}, inplace = True)
df_spp_min.rename(columns = {'년도' : '년도min'}, inplace = True)
df_spp_min

,회사ID,년도min,min년도_연매출액/월평균연금보험료
0,무신사,2015,5.56
1,쏘카,2015,4.62
2,야놀자,2015,1.91
3,에이프로젠,2015,0.20
4,엘앤피코스메틱,2015,15.14
5,옐로모바일,2015,61.57
6,우아한형제들,2015,0.17
7,지피클럽,2015,30.41
8,쿠팡,2015,1.55
9,크래프톤,2015,1.32


In [146]:
#기업별 연도별_월평균보험료 변화량 추출
df_spp_diff=pd.merge(df_spp_min,df_spp_max)
df_spp_diff['변화비율']=df_spp_diff['min년도_연매출액/월평균연금보험료']/df_spp_diff['max년도_연매출액/월평균연금보험료']
df_spp_diff=df_spp_diff.drop(['년도min','년도max','min년도_연매출액/월평균연금보험료','max년도_연매출액/월평균연금보험료'],axis=1)
df_spp_diff

,회사ID,변화비율
0,무신사,0.95
1,쏘카,0.77
2,야놀자,1.21
3,에이프로젠,0.21
4,엘앤피코스메틱,3.02
5,옐로모바일,0.66
6,우아한형제들,0.44
7,지피클럽,0.80
8,쿠팡,0.24
9,크래프톤,13.01


In [148]:
df_spp_dif=pd.merge(df_spp_diff,df_period)
df_spp_dif['기간']=df_spp_dif['기간'].astype(float)
df_spp_dif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사ID    11 non-null     object 
 1   변화비율    11 non-null     float64
 2   기간      11 non-null     float64
dtypes: float64(2), object(1)
memory usage: 352.0+ bytes


In [166]:
df_spp_dif['CAGR']=(df_spp_dif['변화비율'])**(1/df_spp_dif['기간'])-1
df_spp_cagr=df_spp_dif
df_spp_cagr

,회사ID,변화비율,기간,CAGR
0,무신사,0.95,4.00,-0.01
1,쏘카,0.77,4.00,-0.06
2,야놀자,1.21,4.00,0.05
3,에이프로젠,0.21,4.00,-0.32
4,엘앤피코스메틱,3.02,4.00,0.32
5,옐로모바일,0.66,4.00,-0.10
6,우아한형제들,0.44,4.00,-0.18
7,지피클럽,0.80,4.00,-0.06
8,쿠팡,0.24,4.00,-0.30
9,크래프톤,13.01,4.00,0.90


**2.최종산출물**

**2.1 유니콘으로 인정받은 시점의 연금보험료의 최소값**

> **지피클럽 3796700.42**

**2.2. 기업별 기간내 연금보험료 평균 상승율**

In [75]:
df_pen_cagr.sort_values(by='CAGR',ascending=False)

,회사ID,변화량,기간,CAGR
5,옐로모바일,1.93,4.00,0.18
8,쿠팡,0.67,4.00,-0.09
3,에이프로젠,0.49,4.00,-0.16
4,엘앤피코스메틱,0.35,4.00,-0.23
9,크래프톤,0.29,4.00,-0.26
1,쏘카,0.23,4.00,-0.31
6,우아한형제들,0.20,4.00,-0.33
0,무신사,0.16,4.00,-0.36
2,야놀자,0.16,4.00,-0.36
10,토스,0.07,4.00,-0.48


> **지피클럽 -0.50**

**2.3. 유니콘으로 인정받은 시점의 매출/연금보험료 최소값**

> **우아한 형제들 0.17**

**2.4. 기업별 기간내 매출/연금보험료 상승율**

In [167]:
df_spp_cagr.sort_values(by='CAGR',ascending=False)

,회사ID,변화비율,기간,CAGR
9,크래프톤,13.01,4.00,0.90
4,엘앤피코스메틱,3.02,4.00,0.32
2,야놀자,1.21,4.00,0.05
0,무신사,0.95,4.00,-0.01
7,지피클럽,0.80,4.00,-0.06
1,쏘카,0.77,4.00,-0.06
5,옐로모바일,0.66,4.00,-0.10
6,우아한형제들,0.44,4.00,-0.18
8,쿠팡,0.24,4.00,-0.30
3,에이프로젠,0.21,4.00,-0.32


> **토스 -0.67**